# Imports

In [1]:
from numpy.random import seed
seed(111)
from tensorflow import set_random_seed
set_random_seed(222)

#from numpy.random import seed
#seed(404)
#import tensorflow
#tensorflow.random_set_seed(888)

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf
from time import strftime
import tensorboard
import os
from PIL import Image

# Constants

In [3]:

x_train_path='digit_xtrain.csv'
x_test_path='digit_xtest.csv'
y_train_path='digit_ytrain.csv'
y_test_path='digit_ytest.csv'
no_classes=10
LOGGING_PATH="tensorboard_mnist_digit_logs/"



NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

# Data Importing

In [4]:
#loding data into numpy arrays
y_train_all=np.loadtxt(y_train_path,delimiter=',',dtype=int)
y_test1=np.loadtxt(y_test_path,delimiter=',',dtype=int)
x_train_all=np.loadtxt(x_train_path,delimiter=',',dtype=int)
x_test1=np.loadtxt(x_test_path,delimiter=',',dtype=int)

# Exploring Data

In [5]:
x_train_all.shape

(60000, 784)

In [6]:
y_train_all.shape

(60000,)

In [7]:
x_test1.shape

(10000, 784)

In [8]:
y_test1.shape

(10000,)

In [9]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

In [10]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

# Data Pre Processing

RE-SCALING FEATURES

In [11]:
x_train_all, x_test = x_train_all / 255.0, x_test1 / 255.0

CONVERTING LABELS INTO ONE HOT CODING

In [12]:
y_test=np.eye(no_classes)[y_test1]
y_train_all=np.eye(no_classes)[y_train_all]

In [13]:
y_train_all.shape

(60000, 10)

In [14]:
y_test.shape

(10000, 10)

CREATING VALIDATION DATA SET FROM TRAINING DATASET

In [15]:
# creating validation data set from training dataset(10000:50000)
x_val=x_train_all[:10000]
x_train=x_train_all[10000:]
y_val=y_train_all[:10000]
y_train=y_train_all[10000:]


# Set Up Tensorflow Graph

IMPORTS 

In [16]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [17]:
##tensor setup
x=tf.placeholder(dtype=tf.float32,shape=[None,784],name='x')

In [18]:
y=tf.placeholder(tf.float32,shape=[None,NR_CLASSES ],name='Labels')

# Neural Network Architecture

### HYPER PARAMETERS

In [19]:
nr_epochs=20
learning_rate=0.001
total_inputs=784
n_hidden_1=512
n_hidden_2=64
NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

In [20]:
#name_scope is used to group all the layer elements(w,b,activation) in graph
with tf.name_scope('hidden_1'):
    initial_w1=tf.truncated_normal(shape=[784,512],stddev=0.1,seed=42)
    w1=tf.Variable(initial_value=initial_w1,name='w1')
    initial_b1=tf.constant(value=0.0,shape=[512])
    b1=tf.Variable(initial_value=initial_b1,name='b1')
    layer1_in=tf.matmul(x,w1)+b1
    layer1_out=tf.nn.relu(layer1_in)
    tf.summary.histogram('weights',w1)
    tf.summary.histogram('biases',b1)

In [21]:
with tf.name_scope('hidden_2'):

    initial_w2=tf.truncated_normal(shape=[512,64],stddev=0.1,seed=42)
    w2=tf.Variable(initial_value=initial_w2,name='w2')
    initial_b2=tf.constant(value=0.0,shape=[64])
    b2=tf.Variable(initial_value=initial_b2,name='b2')
    layer2_in=tf.matmul(layer1_out,w2)+b2
    layer2_out=tf.nn.relu(layer2_in)
    
    tf.summary.histogram('weights',w2)
    tf.summary.histogram('biases',b2)

In [22]:
with tf.name_scope('output_layer'):

    initial_w3=tf.truncated_normal(shape=[64,10],stddev=0.1,seed=42)
    w3=tf.Variable(initial_value=initial_w3,name='w3')
    initial_b3=tf.constant(value=0.0,shape=[10])
    b3=tf.Variable(initial_value=initial_b3,name='b3')
    layer3_in=tf.matmul(layer2_out,w3)+b3
    output=tf.nn.softmax(layer3_in)
    
    tf.summary.histogram('weights',w3)
    tf.summary.histogram('biases',b3)

#instead of defining each hidden layer individually we will define function so that we can use it when ever necessary

def setup_layer(input,weight_dim,bias_dim,name):
    with tf.name_scope(name):
        initial_w=tf.truncated_normal(shape=weight_dim,stddev=0.1,seed=42)
        w=tf.Variable(initial_value=initial_w,name='W')
        initial_b=tf.constant(value=0.0,shape=bias_dim)
        b=tf.Variable(initial_value=initial_b,name='B')
        layer_in=tf.matmul(input,w)+b
        if name=='output':
             layer_out=tf.nn.softmax(layer_in)
        else:
             layer_out=tf.nn.relu(layer_in)
            
        tf.summary.histogram('weights',w)
        tf.summary.histogram('biases',b)
        return layer_out

layer_1=setup_layer(x,weight_dim=[TOTAL_INPUTS,n_hidden_1],
                   bias_dim=[n_hidden_1],name='layer_1')

layer_2=setup_layer(layer_1,weight_dim=[n_hidden_1,n_hidden_2],
                   bias_dim=[n_hidden_2],name='layer_2')
 
output=setup_layer(layer_2,weight_dim=[n_hidden_2,NR_CLASSES],
                   bias_dim=[NR_CLASSES],name='output')

model_name=f'{n_hidden_1}-{n_hidden_2} LR {learning_rate} E{nr_epochs}'

In [24]:
model_name=f'{n_hidden_1}-{n_hidden_2} LR {learning_rate} E{nr_epochs}'

# TENSORBOARD SETUP

In [25]:
##TENSOR BOARD FLODER
folder_name= f'{model_name}_at_{strftime("%H_%M")}'
directory=os.path.join(LOGGING_PATH,folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('SUCESSFULLY CREATED DIRECTORIES')


SUCESSFULLY CREATED DIRECTORIES


# LOSS,OPTIMISATION & METRICS

DEFINING LOSS FUNCTION

In [26]:
with tf.name_scope('loss_calc'):
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=output))

#DEFINING LOSS FUNCTION
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=output))

#DEFINING OPTIMIZER
optimizer=tf.train.AdamOptimizer(learning_rate)
train_step=optimizer.minimize(loss)

#ACCURACY METRICS
model_prediction=tf.argmax(output,axis=1,name='prediction')
correct_pred=tf.equal(model_prediction,tf.argmax(y,axis=1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))


DEFINING OPTIMIZER

In [27]:
with tf.name_scope('optimizer'):
    optimizer=tf.train.AdamOptimizer(learning_rate)
    train_step=optimizer.minimize(loss)

ACCURACY METRICS

In [28]:
with tf.name_scope('accuracy_calc'):
    model_prediction=tf.argmax(output,axis=1,name='prediction')
    correct_pred=tf.equal(model_prediction,tf.argmax(y,axis=1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))


#summary for accuracy and loss

In [29]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy',accuracy)
    tf.summary.scalar('Loss',loss)

CHECK INPUT IMAGES IN TENSORBOARD

In [30]:
with tf.name_scope('show_image'):
    x_image=tf.reshape(x,[-1,28,28,1])
    tf.summary.image('image_input',x_image,max_outputs=4)

# Run Session

In [31]:
sess=tf.Session()


In [32]:
## SETup FILEWRITER and MERGE Summaries
merged_summary=tf.summary.merge_all()

###TRAINING WRITER
train_writer=tf.summary.FileWriter(directory+'/train')
train_writer.add_graph(sess.graph)

###VALIDATION WRITER
validation_writer=tf.summary.FileWriter(directory+'/validation')


# ###initialize

In [33]:

init=tf.global_variables_initializer()
sess.run(init)

In [34]:
size_of_batch=1000
num_examples=y_train.shape[0]
nr_iterations=int(num_examples/size_of_batch)
index_in_epoch=0

def next_batch(batch_size,data,labels):
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start=0
        index_in_epoch=batch_size
    
    end=index_in_epoch
    
    return data[start:end] ,labels[start:end]


# TRAINING_loop

In [35]:
# TRAINING_loop
    
for epoch in range (nr_epochs):
    
    ####TRAINING DATA SET#####
    for i in range (nr_iterations):
        batch_x,batch_y=next_batch(size_of_batch,data=x_train,labels=y_train)
        feed_dictionary = {x:batch_x, y:batch_y}
        sess.run(train_step,feed_dict=feed_dictionary)
    s,batch_accuracy=sess.run(fetches=[merged_summary,accuracy],feed_dict=feed_dictionary)
    train_writer.add_summary(s,epoch)
    print(f'Epoch {epoch} \t|Training Accuracy ={batch_accuracy}')
    
    ########VALIDATION DATA SET######
    summary=sess.run(fetches=merged_summary,feed_dict={x:x_val,y:y_val})
    validation_writer.add_summary(summary,epoch)
    
print("completed>>>>")


Epoch 0 	|Training Accuracy =0.7760000228881836
Epoch 1 	|Training Accuracy =0.7850000262260437
Epoch 2 	|Training Accuracy =0.8769999742507935
Epoch 3 	|Training Accuracy =0.9629999995231628
Epoch 4 	|Training Accuracy =0.9739999771118164
Epoch 5 	|Training Accuracy =0.9800000190734863
Epoch 6 	|Training Accuracy =0.984000027179718
Epoch 7 	|Training Accuracy =0.9879999756813049
Epoch 8 	|Training Accuracy =0.9879999756813049
Epoch 9 	|Training Accuracy =0.9879999756813049
Epoch 10 	|Training Accuracy =0.9860000014305115
Epoch 11 	|Training Accuracy =0.9869999885559082
Epoch 12 	|Training Accuracy =0.9890000224113464
Epoch 13 	|Training Accuracy =0.9900000095367432
Epoch 14 	|Training Accuracy =0.9900000095367432
Epoch 15 	|Training Accuracy =0.9900000095367432
Epoch 16 	|Training Accuracy =0.9909999966621399
Epoch 17 	|Training Accuracy =0.9909999966621399
Epoch 18 	|Training Accuracy =0.9909999966621399
Epoch 19 	|Training Accuracy =0.9909999966621399
completed>>>>


##SAVING MODEL

outputs={'accuracy_calc/prediction':model_prediction}
inputs={'X':x}
tf.saved_model.simple_save(sess,'SavedModel1993',inputs,outputs)

# Make Prediction

In [36]:
#from PIL import Image
img=Image.open('test_img.png')

img

In [37]:
bw=img.convert('L')

In [38]:
bw

In [39]:
img_array=np.invert(bw)

In [40]:
img_array.shape

(28, 28)

In [41]:
#to flaten the array data
test_img=img_array.ravel()

In [42]:
test_img.shape

(784,)

In [43]:
prediction=sess.run(fetches=tf.argmax(output,axis=1),feed_dict={x:[test_img]})

In [44]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


MEAN SQUARE ERROR

In [45]:
from sklearn.metrics import mean_squared_error 
prediction1=sess.run(fetches=tf.argmax(output,axis=1),feed_dict={x:x_test})

MSE=mean_squared_error(y_test1,prediction1) 
print(MSE)

0.4709


# Testing and Evaluation

In [46]:
test_accuracy=sess.run(fetches=accuracy,feed_dict={x:x_test,y:y_test})
            

In [47]:
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 97.42%


##RESET FOR THE NEXT RUN

In [48]:


train_writer.close() ####FOR TRAINING DATA SET
sess.close()
tf.reset_default_graph()


validation_writer.close()  #### FOR VALIDATION DATA SET